In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

**Dataset**

In [ ]:
# Heart disease recognition | پروژه تشخیص بیماری قلبی
# توی دیتاست، 1 یعنی دارای بیماری قلبی و 0 یعنی فاقد بیماری قلبی

# Load the dataset | لود کردن دیتاست
df = pd.read_csv('heart.csv')

# خوندن 5 تا اولین نمونه ها
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [ ]:
# Display dataset info | دیدن اطلاعات کلی دیتاست
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [ ]:
# Display dataset's feature information | دیدن اطلاعات مربوط به فیچر های دیتاست
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


**Pre-processing**

In [ ]:
# Split features and labels | جدا کردن فیچر ها و لیبل ها

X = df.drop('HeartDisease', axis='columns')
y = df['HeartDisease']

X.shape, y.shape

((918, 11), (918,))

In [ ]:
# اینجا ما یه مشکلی داریم
# اینکه بعضی فیچر های دیتاست مثل جنسیت، عدد نیستن که بتونیم نرمالیزه کنیم
# پس از انکدر استفاده میکنیم تا اونا هم به عدد تبدیل شن
# Convert non-numeric features to numeric features to normilize

X = X.apply(LabelEncoder().fit_transform)

# یا به صورت دستی
# df['Sex'] = df['Sex'].replace({'M': 1, 'F': 0})

X.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,12,1,1,41,147,0,1,98,0,10,2
1,21,0,2,55,40,0,1,82,0,20,1
2,9,1,1,31,141,0,2,25,0,10,2
3,20,0,0,39,72,0,1,34,1,25,1
4,26,1,2,49,53,0,1,48,0,10,2


In [ ]:
# Split train and test data | جدا کردن داده های ترِین و تست

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train.shape, X_test.shape

((734, 11), (184, 11))

In [ ]:
# تبدیل داده ها به لیست
# چون الان دیتاست مون پانداسی هست و ما میخوایم اونو به نامپای که راحت تر و سریعتر هست، تغییر بدیم
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# اگر به لیست تبدیل نکنیم باید اینجوری نمونه هارو بخونیم
# X_train.iloc[0]

X_train[0], y_train[0]

(array([ 7,  1,  0, 22, 56,  0,  1, 56,  1, 26,  1]), np.int64(1))

In [ ]:
# Normalize | نرمالیزه کردن داده ها

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train[0], X_test[0]

(array([0.14285714, 1.        , 0.        , 0.32307692, 0.25339367,
        0.        , 0.5       , 0.47457627, 1.        , 0.5       ,
        0.5       ]),
 array([0.36734694, 1.        , 0.        , 0.2       , 0.43438914,
        0.        , 1.        , 0.55932203, 1.        , 0.38461538,
        0.5       ]))

**Model definition**

In [ ]:
# Create the model | ساخت مدل

# Define the model | تعریف مدل
def create_model(optimizer='adam', dropout_rate=0.3, units=128):
  model = keras.Sequential()

  # Input layer | تعریف لایه ورودی
  # که اختیاریه و میشد تو لایه پنهان اول اینپوت شیپ رو نوشت و این خط رو پاک کرد
  model.add(keras.layers.Input(shape=(11,)))

  # hidden layer | تعریف لایه های میانی
  model.add(keras.layers.Dense(units=units, activation='relu'))
  model.add(keras.layers.Dropout(dropout_rate)) # هر بار چند درصد نورون ها رو به صورت شانسی توی این لایه حذف میکنیم تا پیچیدگی مدل بیاد پایین و از اورفیت کردن جلوگیری بشه

  model.add(keras.layers.Dense(units=units, activation='relu'))
  model.add(keras.layers.Dropout(dropout_rate))

  model.add(keras.layers.Dense(units=units, activation='relu'))
  model.add(keras.layers.Dropout(dropout_rate))

  model.add(keras.layers.Dense(units=units, activation='relu'))
  model.add(keras.layers.Dropout(dropout_rate))

  # Output layer | تعریف لایه خروجی
  # خروجی اگه بالای 0.5 باشه یعنی جواب میشه 1 و درغیر این صورت میشه 0
  model.add(keras.layers.Dense(units=1, activation='sigmoid'))

  # Compile the model | کامپایل کردن مدل
  model.compile(optimizer=optimizer, loss=tf.losses.binary_crossentropy, metrics=['accuracy'])

  return model

**Callback**

In [12]:
# کال بک

# اگر به مدت 15 ایپاک متوالی تابع هزینه پایین نیاد، توقف زود هنگام شروع میشه و بعد از توقف بهترین ایپاک انتخاب میشه
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

**Grid search & Random search**

In [ ]:
# Create the model using scikeras to contain occuring errors | ساخت مدل با سایکراس
model = KerasClassifier(model=create_model, epochs=100, batch_size=64, verbose=2, callbacks=[early_stop])

params = {
    "model__optimizer": ["adam", "rmsprop"],
    "model__dropout_rate": [0.2, 0.3, 0.4],
    "model__units": [64, 128],
    "batch_size": [32, 64, 128],
}

# تست ترکیب پارامتر ها

# grid = GridSearchCV(model, param_grid=params, cv=5)
random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=7, cv=5)

# Fit the model | آموزش مدل روی داده های ترِین
random_result = random_search.fit(X_train, y_train)

# Display best parameters and score | نمایش بهترین امتیاز و پارامتر ها
print("Best score : ", random_result.best_score_)
print("Best params : ", random_result.best_params_)

Epoch 1/100
10/10 - 6s - 604ms/step - accuracy: 0.5809 - loss: 0.6686
Epoch 2/100
10/10 - 0s - 10ms/step - accuracy: 0.6610 - loss: 0.6165
Epoch 3/100


c:\Users\Abolfazl\Desktop\Cumputer\programing\machine_learning\AI-Learning-Journey\venv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


10/10 - 0s - 12ms/step - accuracy: 0.7445 - loss: 0.5447
Epoch 4/100
10/10 - 0s - 10ms/step - accuracy: 0.8143 - loss: 0.4753
Epoch 5/100
10/10 - 0s - 11ms/step - accuracy: 0.8416 - loss: 0.4120
Epoch 6/100
10/10 - 0s - 11ms/step - accuracy: 0.8313 - loss: 0.4304
Epoch 7/100
10/10 - 0s - 10ms/step - accuracy: 0.8467 - loss: 0.4179
Epoch 8/100
10/10 - 0s - 20ms/step - accuracy: 0.8416 - loss: 0.4132
Epoch 9/100
10/10 - 0s - 12ms/step - accuracy: 0.8399 - loss: 0.4040
Epoch 10/100
10/10 - 0s - 16ms/step - accuracy: 0.8586 - loss: 0.3860
Epoch 11/100
10/10 - 0s - 18ms/step - accuracy: 0.8620 - loss: 0.3630
Epoch 12/100
10/10 - 0s - 12ms/step - accuracy: 0.8484 - loss: 0.3715
Epoch 13/100
10/10 - 0s - 18ms/step - accuracy: 0.8518 - loss: 0.3568
Epoch 14/100
10/10 - 0s - 11ms/step - accuracy: 0.8552 - loss: 0.3701
Epoch 15/100
10/10 - 0s - 12ms/step - accuracy: 0.8569 - loss: 0.3678
Epoch 16/100
10/10 - 0s - 16ms/step - accuracy: 0.8552 - loss: 0.3406
Epoch 17/100
10/10 - 0s - 12ms/step - a

c:\Users\Abolfazl\Desktop\Cumputer\programing\machine_learning\AI-Learning-Journey\venv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


Epoch 4/100
10/10 - 0s - 9ms/step - accuracy: 0.7922 - loss: 0.4902
Epoch 5/100
10/10 - 0s - 9ms/step - accuracy: 0.8126 - loss: 0.4580
Epoch 6/100
10/10 - 0s - 8ms/step - accuracy: 0.8109 - loss: 0.4510
Epoch 7/100
10/10 - 0s - 13ms/step - accuracy: 0.8058 - loss: 0.4415
Epoch 8/100
10/10 - 0s - 10ms/step - accuracy: 0.8399 - loss: 0.4199
Epoch 9/100
10/10 - 0s - 16ms/step - accuracy: 0.8399 - loss: 0.4202
Epoch 10/100
10/10 - 0s - 16ms/step - accuracy: 0.8348 - loss: 0.4270
Epoch 11/100
10/10 - 0s - 10ms/step - accuracy: 0.8365 - loss: 0.4218
Epoch 12/100
10/10 - 0s - 11ms/step - accuracy: 0.8313 - loss: 0.4167
Epoch 13/100
10/10 - 0s - 11ms/step - accuracy: 0.8348 - loss: 0.3986
Epoch 14/100
10/10 - 0s - 8ms/step - accuracy: 0.8365 - loss: 0.4082
Epoch 15/100
10/10 - 0s - 8ms/step - accuracy: 0.8279 - loss: 0.4000
Epoch 16/100
10/10 - 0s - 7ms/step - accuracy: 0.8382 - loss: 0.3844
Epoch 17/100
10/10 - 0s - 8ms/step - accuracy: 0.8245 - loss: 0.3898
Epoch 18/100
10/10 - 0s - 8ms/ste

...

Epoch 4/100
10/10 - 0s - 10ms/step - accuracy: 0.7956 - loss: 0.4624
Epoch 5/100
10/10 - 0s - 11ms/step - accuracy: 0.8160 - loss: 0.4476
Epoch 6/100
10/10 - 0s - 17ms/step - accuracy: 0.8228 - loss: 0.4446
Epoch 7/100
10/10 - 0s - 10ms/step - accuracy: 0.8399 - loss: 0.4081
Epoch 8/100
10/10 - 0s - 11ms/step - accuracy: 0.8160 - loss: 0.4627
Epoch 9/100
10/10 - 0s - 12ms/step - accuracy: 0.8382 - loss: 0.4119
Epoch 10/100
10/10 - 0s - 15ms/step - accuracy: 0.8296 - loss: 0.4045
Epoch 11/100
10/10 - 0s - 40ms/step - accuracy: 0.8348 - loss: 0.4107
Epoch 12/100
10/10 - 0s - 13ms/step - accuracy: 0.8382 - loss: 0.4027
Epoch 13/100
10/10 - 0s - 12ms/step - accuracy: 0.8399 - loss: 0.3900
Epoch 14/100
10/10 - 0s - 15ms/step - accuracy: 0.8433 - loss: 0.4049
Epoch 15/100
10/10 - 0s - 9ms/step - accuracy: 0.8416 - loss: 0.3969
Epoch 16/100
10/10 - 0s - 9ms/step - accuracy: 0.8365 - loss: 0.3882
Epoch 17/100
10/10 - 0s - 9ms/step - accuracy: 0.8330 - loss: 0.4002
Epoch 18/100
10/10 - 0s - 9ms

c:\Users\Abolfazl\Desktop\Cumputer\programing\machine_learning\AI-Learning-Journey\venv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


10/10 - 0s - 11ms/step - accuracy: 0.7670 - loss: 0.5234
Epoch 4/100
10/10 - 0s - 9ms/step - accuracy: 0.7874 - loss: 0.4835
Epoch 5/100
10/10 - 0s - 9ms/step - accuracy: 0.8078 - loss: 0.4489
Epoch 6/100
10/10 - 0s - 9ms/step - accuracy: 0.8180 - loss: 0.4230
Epoch 7/100
10/10 - 0s - 11ms/step - accuracy: 0.8384 - loss: 0.4207
Epoch 8/100
10/10 - 0s - 10ms/step - accuracy: 0.8486 - loss: 0.3993
Epoch 9/100
10/10 - 0s - 13ms/step - accuracy: 0.8418 - loss: 0.3939
Epoch 10/100
10/10 - 0s - 15ms/step - accuracy: 0.8401 - loss: 0.3970
Epoch 11/100
10/10 - 0s - 9ms/step - accuracy: 0.8401 - loss: 0.3942
Epoch 12/100
10/10 - 0s - 12ms/step - accuracy: 0.8367 - loss: 0.3786
Epoch 13/100
10/10 - 0s - 11ms/step - accuracy: 0.8316 - loss: 0.3833
Epoch 14/100
10/10 - 0s - 8ms/step - accuracy: 0.8520 - loss: 0.3755
Epoch 15/100
10/10 - 0s - 10ms/step - accuracy: 0.8537 - loss: 0.3761
Epoch 16/100
10/10 - 0s - 8ms/step - accuracy: 0.8503 - loss: 0.3724
Epoch 17/100
10/10 - 0s - 8ms/step - accuracy

c:\Users\Abolfazl\Desktop\Cumputer\programing\machine_learning\AI-Learning-Journey\venv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


6/6 - 0s - 19ms/step - accuracy: 0.5654 - loss: 0.6732
Epoch 4/100
6/6 - 0s - 25ms/step - accuracy: 0.5790 - loss: 0.6597
Epoch 5/100
6/6 - 0s - 14ms/step - accuracy: 0.5790 - loss: 0.6426
Epoch 6/100
6/6 - 0s - 16ms/step - accuracy: 0.5913 - loss: 0.6318
Epoch 7/100
6/6 - 0s - 31ms/step - accuracy: 0.6158 - loss: 0.5998
Epoch 8/100
6/6 - 0s - 14ms/step - accuracy: 0.6744 - loss: 0.5939
Epoch 9/100
6/6 - 0s - 14ms/step - accuracy: 0.7302 - loss: 0.5675
Epoch 10/100
6/6 - 0s - 20ms/step - accuracy: 0.7589 - loss: 0.5389
Epoch 11/100
6/6 - 0s - 14ms/step - accuracy: 0.8038 - loss: 0.5067
Epoch 12/100
6/6 - 0s - 13ms/step - accuracy: 0.8038 - loss: 0.4938
Epoch 13/100
6/6 - 0s - 14ms/step - accuracy: 0.8093 - loss: 0.4575
Epoch 14/100
6/6 - 0s - 12ms/step - accuracy: 0.8188 - loss: 0.4595
Epoch 15/100
6/6 - 0s - 15ms/step - accuracy: 0.8079 - loss: 0.4738
Epoch 16/100
6/6 - 0s - 13ms/step - accuracy: 0.8174 - loss: 0.4522
Epoch 17/100
6/6 - 0s - 14ms/step - accuracy: 0.8283 - loss: 0.4245